# Bootcomp  - selection of top m of k simulated systems

This Jupyter _Python 3_ notebook has been written to accompany the WSC18 paper:

**PRACTICAL CONSIDERATIONS IN SELECTING THE BEST SET OF SIMULATED SYSTEMS**  _by Christine Currie and Tom Monks.

The notebook provides a worked example of using BootComp to conduct a 2 stage screening and search of a simulation model.    It is a simple example that does not consider chance constraints.  Note that Bootcomp would typically be used in a multiple KPI situation.



## 1. Preamble

### 1.1. Detail of the simulation model

The simulation model was used in a 2017 project in the UK to help a hospital, a community healthcare provider and a clinical commissioning group design and plan a new community rehabilitation ward.  In the UK, patients who require rehabilitation are often stuck in a queuing system where there must wait (inappropriately) in a acute hospital bed for a space in the rehabilitaiton ward.  The model investigated the sizing of the new ward in order to minimise patient waiting time whilst meeting probabilitic constraints regarding ward occupancy (bed utilization) and the number of transfers between single sex bays.

<img src="images/DToC.jpg" alt="Delayed Transfers of Care Model" title="Simulation Model and KPIs" />

### 1.2. Output data

The output data for the example analysis are bundled with git repository.  There are three .csv files in the data/ directory for 'waiting times', 'utilization' and 'transfers'.  

The model itself is not needed.  There are 50 replications of 1151 competing designs points.  Users can vary the number of replications used in the two stage procedure.  

The experimental design is also included for reference.

## 2. Prerequisites

### 2.1. BootComp Modules

In [1]:
import Bootstrap as bs
import BootIO as io
import ConvFuncs as cf

In [2]:
#WSC18 specific
import Bootstrap_crn as crn
from Bootstrap_crn import bootstrap_chance_constraint

### 2.2. Python Data Science Modules

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import scipy.stats
%matplotlib inline
import matplotlib as mp
import seaborn as sns
from numba import jit

## 3. Procedure: Stage 1

** Optimization Parameters **

N_BOOTS = no. bootstraps to perform

** Stage 1 **

$n_1$ = no. stage 1 indeptendent replications for each systems / competing design

$p_1$ = percentage of bootstrap samples that must meet chance constraint in stage 1

$y_1$ proportion of bootstrap samples of primary KPI that must be within $x_1$ percent of the best system

** Stage 2 **

$n_2$ = no. stage 2 independent replications for each system / competigin design

$p_1$ = percentage of bootstrap samples that must meet chance constraint in stage 2

$y_1$ proportion of bootstrap samples of primary KPI that must be within $x_1$ percent of the best system in stage 2

In [4]:
N_BOOTS = 1000
m = 3
n_1 = 20
n_2 = 480

budget = 2000 # total number of replications

gamma_1 = 0.95  # not relevant with no chance contraints
x_1 = 0.5 
y_1 = 0.8

x1_limits = np.linspace(0.3,0.1,3)#.arange(0.05, 0.5, 0.05)


gamma_2 = 0.95  # not relevant with no chance contraints
x_2 = 0.3
y_2 = 0.90

y1_limits = np.linspace(0.6,0.95,8)#np.arange(0.8, 0.95, 0.05)

print(x1_limits)
print(y1_limits)


[ 0.3  0.2  0.1]
[ 0.6   0.65  0.7   0.75  0.8   0.85  0.9   0.95]


In [5]:
def simulate(file_name, reps):
    return crn.load_systems(file_name, exclude_reps = 10000-reps)
    

In [6]:
def simulate_experiment(model, reps, systems):
    return model[:,:reps][systems]
    

### Step 1: Read in initial $ n_1 $  replications

In [7]:
model_file = "data/EG1a.csv"

In [26]:
def indifferent(x, indifference):
    """
    convert numbers to 0 or 1
    1 = difference less than x
    0 = difference greater than x
    """
    if x > 0:
        return 1
    elif abs(x)<= indifference:
        return 1
    else:
        return 0
    

def stage1_output(diffs, x_1, y_1, systems, best_system_index, nboots):
    indifference = systems[best_system_index].mean() * x_1
    df_indifference = diffs.applymap(lambda x: indifferent(x, indifference))
    #print('x: {0}; y:{1}'.format(x_1, y_1))
    
    #if(x_1 == 0.1 and y_1 >= 0.94):
    #    print(diffs)
    #    print(df_indifference)
    threshold = nboots * y_1
    df_within_limit = df_indifference.sum(0)
    df_within_limit= pd.DataFrame(df_within_limit, columns=['sum'])
    return df_within_limit.loc[df_within_limit['sum'] >= threshold].index


def bootcomp_run(systems, xlimits, ylimits, nboots, k, m, labels):
    df_wait = pd.DataFrame(systems)
    
    args =  bs.BootstrapArguments()

    args.nboots = nboots
    args.nscenarios = k
    args.point_estimate_func = bs.bootstrap_mean
    
    subset_waits = df_wait.mean()
    subset_waits.rename('wait', inplace=True)
    
    subset_kpi = pd.concat([subset_waits], axis=1)
    best_system_index = subset_kpi.sort_values(by=['wait'], ascending=False).index[0]
    
    feasible_systems = df_wait
    
    #edited here (added .mean())
    diffs =  pd.DataFrame(feasible_systems.values.T - np.array(feasible_systems[best_system_index]).mean()).T
    diffs[best_system_index] = 0
    #print(diffs)                     
    resample_diffs = bs.resample_all_scenarios(diffs.values.T.tolist(), args)
    
    df_boots_diffs= cf.resamples_to_df(resample_diffs, nboots)
    df_boots_diffs.columns = labels

    optim = []

    for x in xlimits:
        for y in ylimits:
            optim.append([x, y, stage1_output(df_boots_diffs, x, y, feasible_systems, best_system_index, nboots).values])
    
    return pd.DataFrame(optim, columns = ['x1', 'y1', 's1 output'])

In [46]:
def test_bootcomp(budget, model, m, k, n = 20, nboots=1000):
    #stage 1
    
    select_systems = [i for i in range(k)]
    data = simulate_experiment(model = model, reps = n, systems = select_systems)
    

    results = bootcomp_run(data.T, x1_limits, y1_limits, nboots, k, m, select_systems)
    #print(results)
    take_forward = crn.auto_select_parameters(results)
    print(take_forward)
    #stage 2
    #equal allocation of remaining budget
    stage2_replicates = int((budget - (n * k))/len(take_forward)) + n
    #print(stage2_replicates)
    data = simulate_experiment(model = model, reps = n, systems = (take_forward - 1).tolist())
    k = data.shape[0]
    
    results = bootcomp_run(data.T, x1_limits, y1_limits, N_BOOTS, k, m, take_forward.tolist())
    print(results)
    return auto_select_top_m(results, m)

In [10]:
@jit(nopython=True)
def bootstrap(data, boots=1000):
    """
       
    Returns a numpy array containing the bootstrap resamples
    Useful for creating a large number of experimental datasets for testing R&S routines
    
    @data = numpy.array of systems to boostrap
    @boots = number of bootstrap (default = 1000)
    """

    experiments = boots
    designs = 10
    samples = data.shape[1]

    datasets = np.zeros((experiments, designs, samples))
     
    for exp in range(experiments):
        
        for design in range(designs):

            for i in range(samples):

                datasets[exp][design][i] = data[design][round(np.random.uniform(0, samples)-1)]
      
    return datasets

In [11]:
def cs(selected_top_m, true_top_m):
    """
    Returns boolean value.  
    True = correct selection of top m
    False = incorrect selection (one or more of selected top m is incorrect)
    
    @selected_top_m - numpy.array containing the indexes of the top m means selected by the algorithm
    @true_top_m - numpy.array containing the indexes of the true top m means
    
    """
    return np.array_equal(np.sort(selected_top_m), true_top_m)

# Create Experimental Data

In [31]:
data = np.genfromtxt(model_file, delimiter=",", skip_footer=0).transpose()
experiments = bootstrap(data, boots = 10)
true_top_m = [7, 8, 9]
T = 3000
budgets = [i for i in range(1000, T + 200, 200)]

In [13]:
def auto_select_top_m(results_stage2, m):
    df = results_stage2
    df['length'] = df['s1 output'].str.len()
    df_m = df.loc[df['length'] == m]
    
    if(df_m.shape[0] == 0):
        df_m = df.loc[df['length'] > m]

    return df_m[-1:]['s1 output'].values[0][-m:]


In [17]:
pd.DataFrame(experiments[1]).T[:20].to_clipboard(excel=True)

In [47]:
x = test_bootcomp(budget=5000, model=experiments[4], n = 100, m=3, k=10, nboots = 1000)
print(x)

[5 7 8 9]
     x1    y1 s1 output
0   0.3  0.60    [8, 9]
1   0.3  0.65    [8, 9]
2   0.3  0.70    [8, 9]
3   0.3  0.75    [8, 9]
4   0.3  0.80    [8, 9]
5   0.3  0.85    [8, 9]
6   0.3  0.90    [8, 9]
7   0.3  0.95    [8, 9]
8   0.2  0.60    [8, 9]
9   0.2  0.65    [8, 9]
10  0.2  0.70    [8, 9]
11  0.2  0.75       [9]
12  0.2  0.80       [9]
13  0.2  0.85       [9]
14  0.2  0.90       [9]
15  0.2  0.95       [9]
16  0.1  0.60       [9]
17  0.1  0.65       [9]
18  0.1  0.70       [9]
19  0.1  0.75       [9]
20  0.1  0.80       [9]
21  0.1  0.85       [9]
22  0.1  0.90       [9]
23  0.1  0.95       [9]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [29]:
def numerical_experiment(experiments, budgets, m, n_0):
    """
    Conduct a user set number of numerical experiments on the algorithm
    for different computational budgets
    
    Returns:
    1. numpy.array containing P{cs} for each budget
        
    @experiments numpy.array[experiments][designs][replication]
    @budgets python.list containing budgets
    @model_file string path to model 
    
    """
    
    n_experiments = experiments.shape[0]
    k = experiments.shape[1]
    
    correct_selections = np.zeros((n_experiments, len(budgets)))
   
    for exp in range(n_experiments):

        for t in range(len(budgets)):
            #print(experiments[exp].shape)
            #try:
            selected_top_m = test_bootcomp(budget=budgets[t], model=experiments[exp],m=m, k=k, n=n_0).tolist()
            #selected_top_m = selected_top_m[0]
            print(selected_top_m)
            correct_selections[exp][t] = cs(selected_top_m, true_top_m)
            #except:
            #    correct_selections[exp][t] = 0
            
            

    return correct_selections
    

note: could do proportional allocation for stage 1 and 2.
e.g. budget = 1000 with 

In [34]:
css = numerical_experiment(experiments, budgets, 3, 50)

[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[7, 8, 9]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
pd.DataFrame(css)

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = pd.DataFrame(css, columns=budgets).mean(axis=0).plot(style='.-', ms=15)
ax.set_ylabel("P{CS}")
ax.set_xlabel("Budget (max no. replications)")
ax.set_xticks(ticks = [i for i in range(200, 8000, 200)])

ax.plot()

### Step 2: Limit to systems that satisfy chance constraints - IGNORE

Bootstrap function arguments

In [ ]:
args =  bs.BootstrapArguments()

args.nboots = N_BOOTS
args.nscenarios = N_SCENARIOS
args.point_estimate_func = bs.bootstrap_mean

#### No Chance constraints

In [ ]:
subset_waits = df_wait.mean()
subset_waits.rename('wait', inplace=True)

List and rank the systems along with their peformance measures

In [ ]:
subset_kpi = pd.concat([subset_waits], axis=1)

In [ ]:
best_system_index = subset_kpi.sort_values(by=['wait'], ascending=False).index[0]

In [ ]:
best_system_index

### Step 3: setup differences

In [ ]:
feasible_systems = df_wait

In [ ]:
diffs =  pd.DataFrame(feasible_systems.values.T - np.array(feasible_systems[best_system_index])).T
diffs

### Step 4: Quality Bootstrap i.e. Simple bootstrap of differences

In [ ]:
resample_diffs = bs.resample_all_scenarios(diffs.values.T.tolist(), args)

In [ ]:
df_boots_diffs= cf.resamples_to_df(resample_diffs, N_BOOTS)
df_boots_diffs.shape

In [ ]:
df_boots_diffs.head()

### Step 6: Rank systems  

In [ ]:

indifference = feasible_systems[best_system_index].mean() * x_1
indifference

In [ ]:


def indifferent(x, indifference):
    """
    convert numbers to 0 or 1
    1 = difference less than x
    0 = difference greater than x
    """
    if abs(x) <= indifference:
        return 1
    else:
        return 0

In [ ]:
df_indifference = df_boots_diffs.applymap(lambda x: indifferent(x, indifference))
df_indifference.mean()

### Step 7: Define set $J$ where y% of bootstraps are within x% of the best mean

In [ ]:

threshold = N_BOOTS * y_1
df_within_limit = df_indifference.sum(0)
df_within_limit= pd.DataFrame(df_within_limit, columns=['sum'])
take_forward = df_within_limit.loc[df_within_limit['sum'] >= threshold].index

In [ ]:
take_forward

In [ ]:
def stage1_output(x_1, y_1, systems, best_system_index):
    indifference = systems[best_system_index].mean() * x_1
    df_indifference = df_boots_diffs.applymap(lambda x: indifferent(x, indifference))
    threshold = N_BOOTS * y_1
    df_within_limit = df_indifference.sum(0)
    df_within_limit= pd.DataFrame(df_within_limit, columns=['sum'])
    return df_within_limit.loc[df_within_limit['sum'] >= threshold].index

In [ ]:
optim = []

for x in x1_limits:
    for y in y1_limits:
        optim.append([x, y, stage1_output(x, y, feasible_systems, best_system_index).values])

df = pd.DataFrame(optim, columns = ['x1', 'y1', 's1 output'])
df['length'] = df['s1 output'].str.len()
df.loc[df['length'] == df['length'].max()][:1]['s1 output'].values.tolist()

In [ ]:
take_forward = stage1_output(0.5, 0.8, feasible_systems, best_system_index).values

In [ ]:
take_forward

In [ ]:
no_stage1 = take_forward.shape[0]

## 4. Procedure - Stage 2

### Step 8: More replicates of promicing solutions using Common Random Numbers

In [ ]:
stage2_replicates = int((budget - (n_1 * N_SCENARIOS)) / len(take_forward)) + n_1


In [ ]:
df_wait_s2 = pd.DataFrame(crn.load_systems(INPUT_DATA1, exclude_reps = 10000-(stage2_replicates)))[np.array(take_forward)-1]

N_SCENARIOS = len(take_forward)
N_REPS = df_wait_s2.shape[0]

print("Loaded waiting time data. {0} systems; {1} independent replications".format(df_wait_s2.shape[1], df_wait_s2.shape[0]))


### Step 9: Repeat steps 2 - 6 from stage 1

#### Step 2 - Chance contraints . Step not applicable.

In [ ]:
def get_subset_kpi(subset):
    subset_waits = df_wait_s2[subset].mean()
    subset_waits.rename('wait', inplace=True)    
    subset_kpi = pd.concat([subset_waits, subset_utils, subset_tran], axis=1)
    subset_kpi.index.rename('System', inplace=True)
    
    return subset_kpi

In [ ]:

subset_waits = df_wait_s2.mean()


In [ ]:
subset_kpi = pd.concat([subset_waits], axis=1)
subset_kpi.index.rename('System', inplace=True)

### Step [?]  Setup differences from best (stage 2)

In [ ]:
feasible_systems = df_wait_s2
diffs =  pd.DataFrame(feasible_systems.values.T - np.array(feasible_systems[best_system_index])).T
diffs.columns = take_forward

### Bootstrap differences

In [ ]:
resample_diffs = bs.resample_all_scenarios(diffs.values.T.tolist(), args)


In [ ]:
df_boots_diffs= cf.resamples_to_df(resample_diffs, args.nboots)
df_boots_diffs.columns = take_forward
df_boots_diffs.shape

In [ ]:
indifference = feasible_systems[best_system_index].mean() * x_2
indifference

In [ ]:
df_indifference = df_boots_diffs.applymap(lambda x: indifferent(x, indifference))
df_indifference.mean()

In [ ]:
threshold = args.nboots * y_2
df_within_limit = df_indifference.sum(0)
df_within_limit= pd.DataFrame(df_within_limit, columns=['sum'])
final_set = df_within_limit.loc[df_within_limit['sum'] >= threshold].index

In [ ]:
final_set

Final set of feasible systems selected from the competing designs

In [ ]:
subset_kpi = get_subset_kpi(final_set)
subset_kpi

In [ ]:
temp = df_doe[df_doe.index.isin(final_set)]
#subset_kpi = subset_kpi.applymap(lambda x: '%.4f' % x)
df_final = pd.concat([temp, subset_kpi], axis=1)
df_final.sort_values(by=['tran', 'util', 'wait'])


In [ ]:
print('No. in final set {0}'.format(df_final.shape[0]))

In [ ]:
print('No. taken forward from stage 1: {0}'.format(no_stage1))

In [ ]:
df_final.to_clipboard(excel=True)

## Charts for paper

In [ ]:
df_doe = pd.read_csv(DESIGN, index_col='System')
df_doe.index -= 1

Utilisation

In [ ]:
temp = df_doe.loc[df_doe['Number of Bays']==0]
#temp.index += 1
subset_waits = df_wait[temp.index].mean()
subset_waits.rename('wait', inplace=True)
subset_utils = df_util[temp.index].mean()
subset_utils.rename('util', inplace=True)
subset_trans = df_tran[temp.index].mean()
subset_trans.rename('tran', inplace=True)



subset_utils_sem = df_util[temp.index].sem()
subset_utils_sem.rename('util_sem', inplace=True)

subset_utils_count = df_util[temp.index].count()
subset_utils_count.rename('n_util', inplace=True)

import scipy as sp
import scipy.stats

subset_kpi = pd.concat([temp, subset_waits, subset_utils, subset_trans, subset_utils_sem, subset_utils_count], axis = 1)
subset_kpi['Waiting Time (hrs)'] = round(subset_kpi['wait']*24, 2)

confidence = 0.95

subset_kpi['hw_95'] = subset_kpi['util_sem'] * sp.stats.t.ppf((1+confidence)/2., subset_kpi['n_util']-1)

#fig = plt.figure()
#ax = fig.add_subplot(111)
fig, axes = plt.subplots(nrows=1, ncols=2, sharey=False)

subset_kpi.sort_values('util').plot(y = 'util', x= 'Number of Singles', figsize=(20, 8), fontsize = 18, 
                                    linewidth=3, legend =False, kind='scatter', ax=axes[1], xticks=[x for x in range(43, 56, 1)], xlim=(42, 56), yerr='hw_95')#, xlim=(70, 92), ylim=(0, 35))
axes[0].set_ylabel('Mean Waiting Time (hrs)', fontsize = 18)
axes[1].set_xlabel('Number of Singles (beds)', fontsize = 18)




subset_kpi.plot('Number of Singles', 'Waiting Time (hrs)', figsize=(20, 8), fontsize = 18, 
                                    linewidth=3, legend =False, kind='line', ms=10, style='o-', ax=axes[0], xlim=(42, 56),
                                    xticks=[x for x in range(43, 56, 1)])
axes[0].set_xlabel('Number of Singles (beds)', fontsize = 18)
axes[1].set_ylabel('Mean Utilization (% beds)', fontsize = 18)
axes[0].grid(True)
axes[1].grid(True)
axes[1].legend(['mean (n=5)','95% Confidence Interval'],fontsize=18)
#plt.tight_layout()

In [ ]:
fig.savefig("chance_constraint_stage1.pdf", format = 'pdf', dpi=600, bbox_inches='tight')

In [ ]:
subset_kpi

Patient transfers between bays of beds

In [ ]:
temp=df_doe.loc[df_doe['Total beds']<=54]
#temp.index += 1
subset_waits = df_wait[temp.index].mean()
subset_waits.rename('wait', inplace=True)
subset_utils = df_tran[temp.index].mean()
subset_utils.rename('tran', inplace=True)

subset_kpi = pd.concat([temp, subset_waits, subset_utils], axis = 1)
subset_kpi['Waiting Time (hrs)'] = round(subset_kpi['wait']*24, 2)

subset_kpi.head()

In [ ]:


means = subset_kpi.groupby(['Size of Bays'])['tran'].mean()
means.rename('mean', inplace=True)
sems = subset_kpi.groupby(['Size of Bays'])['tran'].sem()
sems.rename('sem', inplace=True)
counts = subset_kpi.groupby(['Size of Bays'])['tran'].count()
counts.rename('n', inplace=True)




transfers = pd.concat([means, sems, counts], axis=1)
confidence = 0.95

transfers['hw_95'] = transfers['sem'] * sp.stats.t.ppf((1+confidence)/2., transfers['n']-1)

#fig = plt.figure()
#ax = fig.add_subplot(111)
fig, axes = plt.subplots(nrows=1, ncols=1)
transfers = transfers.loc[transfers.index >0]
transfers = transfers.loc[transfers.index <27]
transfers.plot(y='mean', x=transfers.index, figsize=(20, 8), fontsize = 14, 
                              linewidth=3, legend =False, kind='line', ax=axes, yerr='hw_95'
              , xlim=(2, 27), xticks=[x for x in range(3, 26, 2)])

axes.set_xlabel('Bay Size (beds)', fontsize = 14)

axes.set_ylabel('Mean Bay Transfers', fontsize = 14)

In [ ]:
transfers